# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,11.06,67,40,9.44,SH,1691597483
1,1,yellowknife,62.4560,-114.3525,18.29,55,100,5.66,CA,1691597483
2,2,avarua,-21.2078,-159.7750,22.03,73,75,6.69,CK,1691597483
3,3,atafu village,-8.5421,-172.5159,28.09,80,98,5.91,TK,1691597483
4,4,albany,42.6001,-73.9662,26.43,65,86,0.45,US,1691597483


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
# %%capture --no-display

# Configure the map plot
citymap = city_data_df.hvplot.points('Lng', 
                                    'Lat', 
                                    geo = True, 
                                    color = 'City', 
                                    size = 'Humidity', 
                                    tiles = 'OSM')

# Display the map
citymap


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
# My perfect day 
# 20-26c temp
# low clouds 
# gentile winds

myday = city_data_df[(city_data_df['Max Temp'] < 26)
                    & (city_data_df['Max Temp'] > 20)
                    & (city_data_df['Cloudiness'] < 2)
                    & (city_data_df['Wind Speed'] < 4)]

# Drop any rows with null values
myday.dropna()

# Display sample data
myday

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
90,90,karratha,-20.7377,116.8463,21.24,66,0,1.12,AU,1691597495
148,148,polyarnyy,69.1989,33.4478,23.78,62,1,1.68,RU,1691597507
189,189,arkhangel'sk,64.5401,40.5433,22.88,59,1,1.71,RU,1691597428
226,226,newman,37.3138,-121.0208,21.80,48,0,0.58,US,1691597527
242,242,soligalich,59.0778,42.2864,21.61,70,0,2.31,RU,1691597540
334,334,omsk,55.0000,73.4000,22.94,53,0,2.00,RU,1691597552
358,358,mochudi,-24.4167,26.1500,21.33,19,1,3.34,BW,1691597555
397,397,helena valley west central,46.6629,-112.0604,24.73,55,0,1.54,US,1691597561
402,402,steinbach,50.1667,8.5667,22.96,54,0,3.60,DE,1691597069
489,489,laguna,38.4210,-121.4238,24.06,73,0,3.60,US,1691597572


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = myday[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
90,karratha,AU,-20.7377,116.8463,66,
148,polyarnyy,RU,69.1989,33.4478,62,
189,arkhangel'sk,RU,64.5401,40.5433,59,
226,newman,US,37.3138,-121.0208,48,
242,soligalich,RU,59.0778,42.2864,70,
334,omsk,RU,55.0000,73.4000,53,
358,mochudi,BW,-24.4167,26.1500,19,
397,helena valley west central,US,46.6629,-112.0604,55,
402,steinbach,DE,50.1667,8.5667,54,
489,laguna,US,38.4210,-121.4238,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 100000
params = {'categories': 'accommodation.hotel', 'apiKey': geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle: {latitude}, {longitude}, {radius}'
    params["bias"] = f'proximity: {latitude}, {longitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data it appears I will be camping at all of my ideal weather locations 
hotel_df

Starting hotel search
karratha - nearest hotel: No hotel found
polyarnyy - nearest hotel: No hotel found
arkhangel'sk - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
soligalich - nearest hotel: No hotel found
omsk - nearest hotel: No hotel found
mochudi - nearest hotel: No hotel found
helena valley west central - nearest hotel: No hotel found
steinbach - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
balqash - nearest hotel: No hotel found
tot'ma - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
90,karratha,AU,-20.7377,116.8463,66,No hotel found
148,polyarnyy,RU,69.1989,33.4478,62,No hotel found
189,arkhangel'sk,RU,64.5401,40.5433,59,No hotel found
226,newman,US,37.3138,-121.0208,48,No hotel found
242,soligalich,RU,59.0778,42.2864,70,No hotel found
334,omsk,RU,55.0000,73.4000,53,No hotel found
358,mochudi,BW,-24.4167,26.1500,19,No hotel found
397,helena valley west central,US,46.6629,-112.0604,55,No hotel found
402,steinbach,DE,50.1667,8.5667,54,No hotel found
489,laguna,US,38.4210,-121.4238,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
# %%capture --no-display

# Configure the map plot
hotelmap = hotel_df.hvplot.points('Lng', 
                                'Lat', 
                                geo = True, 
                                color = 'City', 
                                size = 'Humidity', 
                                tiles = 'OSM',
                                hover_cols = ['Hotel Name', 'Country'],
                                frame_width = 700,)

# Display the map
hotelmap

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)